<a href="https://colab.research.google.com/github/g-larios/arXiv_RAG/blob/main/Website_arxiv_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  conda py313

# Query based on author information or date

We build a RAG based on Gemini that retrieves information from arXiv.com based on user-provided criteria, and produces a completion summarizing the main topics appearing in the summaries of the retrieved articles.

This RAG is deployed through a web application by combining Ngrok and Streamlit.

## Installing Packages

In [1]:
# %pip install -q feedparser

%pip install -q langchain
%pip install -q langchain-community
%pip install -qU google-generativeai
%pip install -qU langchain-google-genai

# %pip install --quiet streamlit
# %pip install --quiet streamlit_chat

# %pip install pyngrok

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.0 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Adding Relavant Keys to the Enviornment

In [1]:
import os
import getpass
import pprint
# from pyngrok import ngrok, conf

os.environ["GOOGLE_API_KEY"] = getpass.getpass(
    "Enter your Google AI API key: ")
os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass(
    "Enter your Llama Cloud API key: ")

# print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
# conf.get_default().auth_token = getpass.getpass()

In [3]:
import re

## arXiv Query + Gemini completion

For an author based query, the arXiv API needs

- full_name: author's full name. The expected format is 'first name' + 'middle name' + 'surname', separated with spaces and with middle name possibly null.

- cat: category in (astro-ph, cond-mat, gr-qc, hep-ex, hep-lat, hep-th,hep-ph, math-ph, nlin, nucl-ex, nucl-th, physics, quant-ph, math, CoRR, q-bio, q-fin, stat, eess, econ). See https://arxiv.org/category_taxonomy for details

### Set up the LLM Model and Langchain Chain
Here we will use Gemini model to do our inference and use langchain to create a small prompt pipeline.

In [53]:
# %%writefile StreamlitApp.py
import urllib
import urllib.request

# import feedparser
# import math
# import datetime
# from dateutil.relativedelta import relativedelta

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=50000,
    timeout=None,
    max_retries=5,
    # other params..
)

# Prompt for short paper summaries
prompt_short_paper_summary = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert in analyzing and summarizing academic research papers."
     " You will receive a research paper in JSON format, containing the paper ID, title, publication date, list of authors, and the full LaTeX-extracted content."

     " Your task is to generate a **concise executive summary** (approximately 150-250 words) that captures the key insights of the paper."

     " **Structure the summary as follows:**"
     " 1. **Title & Authors**: Include the title and list of authors."
     " 2. **Abstract**: If available, summarize the abstract succinctly."
     " 3. **Main Contributions**: Highlight the paper’s key findings, innovations, or arguments."
     " 4. **Methodology (if applicable)**: Briefly describe the approach, models, or experiments used."
     " 5. **Results & Conclusion**: Summarize the main results and their significance."

     " **Guidelines:**"
     " - Focus on the most relevant details while avoiding excessive technical jargon."
     " - If formulas, equations, or mathematical details are central to the paper, summarize their purpose rather than presenting them in full."
     " - Keep the summary concise yet informative (150-250 words)."
     " - Do **not** include personal opinions or speculative interpretations."),

    ("user", "{Prompt}\nContext:\n{Context}"),
])

prompt_detailed_paper_summary = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert in analyzing and summarizing academic research papers."
     " You will receive a research paper in JSON format, containing the paper ID, title, publication date, list of authors, and the full LaTeX-extracted content."

     " Your task is to generate a **comprehensive and structured summary** (approximately 500-800 words) that preserves key insights while maintaining clarity."

     " **Structure the summary as follows:**"
     " 1. **Title & Authors**: Include the title and list of authors."
     " 2. **Abstract**: Provide a structured summary of the abstract."
     " 3. **Main Contributions**: Highlight the paper’s key findings, innovations, and major arguments."
     " 4. **Methodology**: Describe the approach, models, algorithms, or experiments used in detail."
     " 5. **Key Equations & Mathematical Insights**: Present important equations in LaTeX format while summarizing their significance."
     " 6. **Results & Discussion**: Explain the findings in depth, referencing figures, tables, and graphs if they provide critical insights."
     " 7. **Conclusion & Implications**: Summarize the main takeaways and potential impact of the research."

     " **Guidelines:**"
     " - Maintain technical depth while ensuring clarity."
     " - Present mathematical equations in LaTeX format when relevant."
     " - If the paper contains figures, tables, or graphs that are crucial, mention their relevance and summarize key insights."
     " - Ensure the summary is **comprehensive yet structured** to enhance readability (500-800 words)."
     " - Do **not** include personal opinions or speculative interpretations."),

    ("user", "{Prompt}\nContext:\n{Context}"),
])

prompt_detailed_paper_summary2 = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert in analyzing and summarizing academic research papers."
     " You will receive a research paper in JSON format, containing the paper ID, title, publication date, list of authors, and the full LaTeX-extracted content."

     " Your task is to generate a **detailed and structured summary** (approximately 500-800 words) that reflects the original paper's **organization**, while summarizing the key insights and important details."

     " The summary should mirror the **section headings and paragraph structure** of the original paper, ensuring that each section is summarized appropriately while retaining the logical flow of the paper."

     " **Guidelines for structuring the summary:**"
     " - Start by summarizing the **background** and motivation of the paper, explaining the problem being addressed and the importance of the research."
     " - Continue with a summary of the **approach, models, or experiments** used in the paper, including any key mathematical equations or formulas in **LaTeX** format."
     " - Highlight the **main findings** of the paper, referencing any significant results, figures, or graphs. Include equations where they are critical to understanding the results."
     " - Summarize the **interpretation of the results**, discussing their implications within the context of the paper’s argument."
     " - Conclude by summarizing the **contributions** and **future directions** presented in the paper."
     " - Ensure that **mathematical equations** are included in LaTeX format if they are important for understanding the paper's insights."
     " - Maintain technical depth, but ensure that the summary remains clear and concise."

     " - The summary should follow the **original structure** of the paper, maintaining section headings and paragraphs as they appear in the document, while ensuring the summary is **comprehensive but concise** (around 500-800 words)."

     " - Do **not** include personal opinions or speculative interpretations."),

    ("user", "{Prompt}\nContext:\n{Context}"),
])

# Simple summary chains
chain_short_paper_summary = prompt_short_paper_summary | llm
chain_detailed_paper_summary = prompt_detailed_paper_summary | llm
chain_detailed_paper_summary2 = prompt_detailed_paper_summary2 | llm

### ArXiv Query

#### Paper Summary

In [7]:
from IPython.display import Markdown, display
import requests
import tarfile
import io

In [65]:
def extract_arxiv_id(url):
    """Extract the arXiv ID from an arXiv URL, handling both new and old formats."""
    match = re.search(r'arxiv.org/(?:abs|pdf)/([\w-]+/\d{7}|\d+\.\d+)', url)
    return match.group(1) if match else None


def fetch_arxiv_source(arxiv_id):
    """Fetch the LaTeX source of an arXiv paper and extract the main content."""
    url = f"https://arxiv.org/e-print/{arxiv_id}"
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        # Read the tar.gz content into memory
        tar_data = io.BytesIO(response.content)
        with tarfile.open(fileobj=tar_data, mode="r:gz") as tar:
            latex_file = {}
            for member in tar.getmembers():
                if member.isfile() and member.name.endswith(".tex"):
                    file_content = tar.extractfile(
                        member).read().decode("utf-8", errors="ignore")
                    latex_file[member.name] = file_content
            return latex_file
    else:
        print(
            f"Failed to fetch source. HTTP Status Code: {response.status_code}")
        return None

# Not using this anymore since we are well within Gemini flash token limits even with
# long (~200 pages) papers containing references in the tex file


def clean_latex(latex_source):
    """Extract and clean text from LaTeX files for better RAG input."""
    combined_text = ""
    for filename, content in latex_source.items():
        # Remove comments
        content = re.sub(r"%.*", "", content)
        # Remove LaTeX commands (basic regex, can be improved)
        content = re.sub(r"\\[a-zA-Z]+(\[[^\]]*\])?(\{[^}]*\})?", "", content)
        # Append cleaned text
        combined_text += f"\n--- {filename} ---\n{content}\n"

    return combined_text.strip()

In [ ]:
arxiv_id = extract_arxiv_id("https://arxiv.org/abs/2412.16795")
latex_source = fetch_arxiv_source(arxiv_id)
# cleaned_text = clean_latex(latex_source)
for filename, file_content in latex_source.items():
    print(type(file_content))
    print(len(file_content))
    print(f"--- {filename} ---\n")
    print(file_content)  # Print the latex file
    print("\n" + "="*80 + "\n")

<class 'str'>
423588
--- Invitation.tex ---

\documentclass[11pt,a4paper]{article}
\usepackage{jheppub,amsmath,  amssymb,slashed,url,bm,textgreek,upgreek}
\usepackage{graphicx}
\usepackage{epstopdf}
\def\t{{ \sf t}} 
%% \stackrel{j}{\to}
\def\kk{k}
\def\Sigmax{{\Sigma^*}}
\def\eO{{\eusm O}}
\def\phif{f}
\def\tt{{\mathfrak t}}
\def\J{\mathcal J}
\def\neg{\negthinspace}
\def\a{a}
\def\dzzt{\D(\t z,z|\theta)}
\def\dzztt{\D(\t z,z|\t\theta,\theta)}
\def\ggamma{\Gamma}
\def\Stt{\h S}
\def\Q{{\mathcal Q}}
\def\Res{{\mathrm {R}}}
\def\X{{\mathcal X}}
\def\SoS{{\mathrm S}}
\def\ds{{\mathrm{ds}}}
\def\sD{\text{{\sf D}}}
\def\aalpha{\alpha}
\def\FF{{\eusm  F}}
\def\sPi{\Phi}
\def\ct{{\cmmib t}}
\def\JJ{{\eusm J}}
\def\s{{s}}
\def\ii{i}

\def\BB{\bm B}
\def\uu{u}
\def\piGSO{{\Pi_{\mathrm{GSO}}}}
\def\tpiGSO{\t{\Pi}_{\mathrm{GSO}}}
\def\UUU{\varPhi}
\def\SSigma{{\varTheta}}
\def\Stigma{{\varSigma}}
\def\btheta{{\bm \theta}}
\def\mm{{\bm m}}
\def\zzeta{{\bm \zeta}}
\def\zizeta{\zeta}
\def\v{{\eurm 

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=4000,
#     chunk_overlap=500
# )

# chunks = text_splitter.split_text(cleaned_text)

In [ ]:
# chunk_summaries = []
# for chunk in chunks:
#     # Get the response from the chain
#     response = chain_paper_summary.invoke(
#         {"Prompt": "Can you summarize the following? ", "Context": chunk})

#     # Access the text from the AIMessage object
#     summary_part = response.content if hasattr(
#         response, 'content') else str(response)

#     chunk_summaries.append(summary_part)

# # Combine all chunk summaries into a final summary
# final_summary = "\n\n".join(chunk_summaries)

In [54]:
chains = [chain_short_paper_summary,
          chain_detailed_paper_summary, chain_detailed_paper_summary2]

In [ ]:
# Get the response from the chain
get_length = int(
    input("Type 0 for short summary, and 1 or 2 for long summary: "))

response = chains[get_length].invoke(
    {"Prompt": "Can you summarize the following? ", "Context": file_content})

# Access the text from the AIMessage object
summary = response.content if hasattr(
    response, 'content') else str(response)

display(Markdown(summary))

**Title & Authors:** Introduction to Black Hole Thermodynamics, Edward Witten

**Abstract:** This article provides a basic introduction to black hole thermodynamics.  It covers Bekenstein's proposal of black hole entropy, Hawking's discovery of black hole radiation and its Rindler space analog (the Unruh effect), the Euclidean approach to black hole thermodynamics, von Neumann entropy and its applications, the Ryu-Takayanagi formula, and the nature of white holes.

**Main Contributions:** The paper offers a pedagogical overview of black hole thermodynamics, connecting classical and quantum perspectives.  It explains Bekenstein's generalized second law, Hawking's derivation of black hole radiation temperature, and the Gibbons-Hawking Euclidean approach.  The role of gray body factors in modifying the simple Hawking radiation calculation is detailed.  The Unruh effect in Rindler space is presented as a simplified model for understanding black hole thermodynamics.  The paper also introduces von Neumann entropy and its relation to thermodynamic entropy, particularly in the context of black hole formation and evaporation.  Finally, it discusses the Ryu-Takayanagi formula for holographic entanglement entropy and explores the concept of white holes.

**Methodology:** The paper employs a combination of methods.  Hawking radiation is explained using a Lorentzian approach, analyzing the behavior of quantum fields near the black hole horizon.  The Euclidean approach is used to compute the black hole entropy by analyzing the periodicity of the Euclidean black hole metric.  The Ryu-Takayanagi formula is derived using a replica trick and path integral arguments.

**Results & Conclusion:** The paper successfully connects various approaches to black hole thermodynamics, highlighting the interplay between classical and quantum aspects.  It demonstrates the thermal nature of black hole radiation and the importance of considering both thermodynamic and von Neumann entropy.  The Ryu-Takayanagi formula is presented as a powerful tool for calculating entanglement entropy in holographic theories, and the concept of white holes is discussed in the context of time-reversed black hole formation.  The paper concludes by emphasizing the ongoing challenges and open questions in the field, particularly concerning the microscopic origin of black hole entropy and the nature of the Page curve.

In [ ]:
get_length = int(
    input("Type 0 for short summary, and 1 or 2 for long summary: "))

response = chains[get_length].invoke(
    {"Prompt": "Can you summarize the following? ", "Context": file_content})

# Access the text from the AIMessage object
summary = response.content if hasattr(
    response, 'content') else str(response)

display(Markdown(summary))

## Introduction to Black Hole Thermodynamics

**1. Title & Authors:**

Introduction to Black Hole Thermodynamics

Edward Witten


**2. Abstract:**

This article provides a basic introduction to black hole thermodynamics.  It begins by explaining Bekenstein's proposal that black holes possess entropy, proportional to the horizon area.  The article then discusses Hawking's discovery of black hole radiation and its simplified analog in Rindler space (the Unruh effect).  The Euclidean approach to black hole thermodynamics is introduced, along with von Neumann entropy and its applications, the Ryu-Takayanagi formula, and the concept of a white hole.


**3. Main Contributions:**

The paper offers a pedagogical overview of black hole thermodynamics, connecting classical and quantum perspectives.  Key contributions include:

*   **A heuristic explanation of Bekenstein's generalized second law:**  The paper explains how Bekenstein proposed assigning entropy to black holes to prevent violations of the second law when matter falls in. The generalized entropy, $S_{gen} = \frac{A}{4G} + S_{out}$, is introduced, where A is the horizon area and $S_{out}$ is the entropy outside the horizon.  The paper discusses the limitations of this approach when considering low-energy photons.

*   **A detailed explanation of Hawking radiation:** The paper provides a detailed explanation of Hawking's discovery that black holes emit thermal radiation at temperature $T_H = \frac{1}{8\pi GM}$, focusing on the late-time behavior of quantum fields near the horizon.  The concept of gray body factors, accounting for the potential barrier around the black hole, is introduced.

*   **The Unruh effect and Rindler space:** The paper uses Rindler space to illustrate the key concept of how the vacuum state appears thermal to an accelerated observer, providing a simplified framework for understanding Hawking radiation.  The thermofield double state is introduced as a purification of the thermal density matrix.

*   **The Euclidean approach to black hole thermodynamics:** The paper explains how Gibbons and Hawking used Euclidean continuation to demonstrate the thermal nature of black holes, showing that the Euclidean Schwarzschild metric is smooth and complete if the Euclidean time is periodic with period $8\pi GM$.  The computation of the Bekenstein-Hawking entropy using the Euclidean action is detailed, including the Gibbons-Hawking-York boundary term.

*   **Von Neumann entropy and the Page curve:** The paper introduces von Neumann entropy as a microscopic measure of entropy, contrasting it with thermodynamic entropy.  The Page curve, describing the entropy of an evaporating black hole, is discussed, highlighting the transition at the Page time where the black hole entropy becomes comparable to the radiation entropy.

*   **The Ryu-Takayanagi formula:** The paper presents a heuristic motivation and a sketch of the derivation of the Ryu-Takayanagi (RT) formula, which relates the entanglement entropy of a boundary region in a conformal field theory to the area of a minimal surface in the bulk AdS spacetime.  The concept of quantum extremal surfaces is introduced as a refinement of the RT formula.


**4. Methodology:**

The paper employs a combination of methods:

*   **Classical General Relativity:**  The paper uses Penrose diagrams and Kruskal-Szekeres coordinates to analyze the causal structure of black hole spacetimes.

*   **Quantum Field Theory in Curved Spacetime:**  Hawking radiation is explained by analyzing the behavior of quantum fields in a fixed black hole background.

*   **Path Integrals:**  The Euclidean approach to black hole thermodynamics relies heavily on path integrals in Euclidean signature.  The replica trick is used to compute von Neumann entropy.

*   **AdS/CFT Correspondence:** The Ryu-Takayanagi formula and its generalizations are discussed within the framework of AdS/CFT duality.


**5. Key Equations & Mathematical Insights:**

*   **Bekenstein-Hawking entropy:** $S = \frac{A}{4G}$

*   **Hawking temperature:** $T_H = \frac{1}{8\pi GM}$

*   **Generalized entropy:** $S_{gen} = \frac{A}{4G} + S_{out}$

*   **Rindler temperature:** $T = \frac{1}{2\pi L}$

*   **Euclidean Schwarzschild periodicity:** $\tilde{t} \cong \tilde{t} + 8\pi GM$

*   **Ryu-Takayanagi formula:** $S_A = \frac{A(\Sigma_A)}{4G}$


**6. Results & Discussion:**

The paper presents a detailed discussion of the thermal nature of black holes, both from the Lorentzian and Euclidean perspectives.  The Unruh effect in Rindler space is used as a simplified model.  The Page curve is presented as a consequence of unitary black hole evaporation.  The Ryu-Takayanagi formula is discussed in the context of AdS/CFT, providing a holographic calculation of entanglement entropy.  The paper also discusses the concept of a white hole as the time-reversed black hole and its implications for the second law of thermodynamics.


**7. Conclusion & Implications:**

The paper provides a comprehensive overview of black hole thermodynamics, bridging classical and quantum perspectives.  It highlights the importance of both thermodynamic and von Neumann entropy in understanding black hole physics.  The discussion of the Page curve and the Ryu-Takayanagi formula emphasizes the role of entanglement in black hole thermodynamics.  The analysis of white holes underscores the irreversibility of black hole formation and the instability of white holes.  The paper's pedagogical approach makes it a valuable resource for understanding this complex and fascinating area of research.

In [40]:
get_length = int(
    input("Type 0 for short summary, and 1 or 2 for long summary: "))

response = chains[get_length].invoke(
    {"Prompt": "Can you summarize the following? ", "Context": file_content})

# Access the text from the AIMessage object
summary = response.content if hasattr(
    response, 'content') else str(response)

display(Markdown(summary))

## Introduction to Black Hole Thermodynamics: A Detailed Summary

This paper by Edward Witten provides a comprehensive introduction to black hole thermodynamics, bridging classical and quantum perspectives.  The introduction highlights the inherent tension between the irreversibility of black hole absorption (nothing escapes the event horizon) and fundamental principles of physics, particularly the time-reversal symmetry implied by a Hermitian Hamiltonian.  The author draws an analogy to everyday irreversible processes, like a broken egg, explaining that while time-reversed processes are theoretically possible, they are exponentially improbable due to entropy increase.  Black hole thermodynamics posits that the irreversibility of black hole absorption is analogous to this statistical irreversibility, with entropy increasing when matter falls into a black hole. The paper promises to cover Bekenstein's proposal of black hole entropy, Hawking's discovery of black hole radiation, the Unruh effect, the Euclidean approach, von Neumann entropy, the Ryu-Takayanagi formula, and the nature of white holes.

### Black Hole Entropy and the Generalized Second Law

The Second Law of Thermodynamics states that entropy in observable processes is non-decreasing. Bekenstein, prompted by John Wheeler, noted that the apparent entropy decrease when matter falls into a black hole (assuming zero black hole entropy) violates this law.  To resolve this, he proposed that black hole entropy is proportional to the horizon area, a quantity that only increases in classical general relativity according to the area theorem. For a Schwarzschild black hole of mass $M$, the horizon area is $A = 16\pi G^2 M^2$, and Bekenstein's formula for black hole entropy was $S = \frac{A}{4G\hbar}$.  The constant of proportionality, $\frac{1}{4}$, was later determined by Hawking.  Bekenstein defined a generalized entropy, $S_{gen} = \frac{A}{4G} + S_{out}$, where $S_{out}$ is the entropy of matter and radiation outside the horizon, and proposed a Generalized Second Law: $\frac{dS_{gen}}{dt} \geq 0$.  The author analyzes the validity of this law when a black hole absorbs black body radiation, showing that it holds as long as the photon wavelength is sufficiently small compared to the black hole radius.  The apparent violation for long wavelengths is resolved by considering black hole emission, a point clarified by Hawking's discovery. The large entropy of macroscopic black holes implies a vast degeneracy of quantum states, seemingly contradicting the classical "no hair theorem," which is reinterpreted here as thermalization.

### Black Hole Evaporation

Hawking's 1974 discovery showed that black holes are not truly black at the quantum level.  The author uses Penrose diagrams to illustrate the causal structure of black hole formation and the limitations of observations made by an outside observer.  Hawking's analysis involves studying a quantum field in a Schwarzschild black hole background, a valid approximation for black holes much heavier than the Planck mass.  The author explains how late-time observations by a distant observer probe the quantum state at exponentially small distances from the horizon, effectively probing the vacuum state.  Due to the diverging redshift near the horizon, these late-time observations correspond to a thermal correlation function at the Hawking temperature, $T_H = \frac{1}{8\pi GM}$, implying that the black hole radiates thermally.  This confirms Bekenstein's entropy formula and explains the constant of proportionality.  The author uses Kruskal-Szekeres coordinates to justify the logarithmic relation between the observer's time and the distance from the horizon.  The huge entropy of a black hole suggests degrees of freedom residing on the horizon, a picture illustrated by Wheeler.  The author discusses the slow rate of black hole evaporation and the puzzle of thermal radiation from a potentially pure initial state, attributing the thermal appearance to the mixed state of quantum fields outside the horizon.

### Gray Body Factors

The simplified derivation of Hawking radiation in the previous section assumes free propagation of signals from the horizon.  This is refined by considering a potential barrier due to angular momentum, leading to reflection and absorption of outgoing signals.  The author analyzes a massless scalar field in a Schwarzschild background, deriving an effective potential that vanishes near the horizon and at infinity, with a barrier in between.  Even for zero angular momentum, a potential barrier exists.  The author argues that the thermal emission rate should be multiplied by the transmission probability across this barrier, which is the same in both directions due to the symmetry of the scattering problem.  A more detailed derivation, tracing measurements back to an initial hypersurface and solving the wave equation backwards in time, incorporates the potential barrier and shows that the emission rate is indeed suppressed by the transmission probability squared (gray body factors).  This suppression is crucial for a finite luminosity.  The author discusses the thermodynamic instability of black hole equilibrium in asymptotically flat spacetime due to negative specific heat and the impossibility of a thermal gas filling all of spacetime.  The analysis is valid in the limit $G \to 0$, where these instabilities disappear.  The author concludes by discussing the stabilization of equilibrium in asymptotically Anti-de Sitter (AdS) spacetime.

### Thermodynamics of Rindler Space

The Hawking effect is explained by considering the vacuum state in Minkowski space from the perspective of an observer in a Rindler wedge.  The author uses path integrals to construct the reduced density matrix for an observer who can only measure fields in one Rindler wedge.  The density matrix is shown to be thermal at inverse temperature $2\pi$, a result interpreted as the Unruh effect.  The author relates this to the Lorentz boost generator and shows that the Minkowski vacuum is the thermofield double state of the Rindler wedges.  The derivation is generalized to arbitrary quantum field theories.

### Euclidean Approach to Black Hole Thermodynamics

The Euclidean approach, due to Gibbons and Hawking, involves continuing the Schwarzschild metric to Euclidean signature by setting $t = -i\tau$.  The resulting metric is smooth and complete if $\tau$ is periodic with period $8\pi GM$, the inverse Hawking temperature.  The author explains the topological structure of the Euclidean Schwarzschild spacetime and proposes that the thermal nature of a black hole reflects this periodicity in imaginary time.  The author computes the Euclidean black hole action, including the Gibbons-Hawking-York boundary term, and uses it to derive the Bekenstein-Hawking entropy formula.  An alternative computation, varying the period of $\tau$ while keeping the mass fixed, introduces a conical singularity at the horizon, whose contribution to the action directly yields the entropy formula.  The author discusses the interpretation of the computed entropy as a counting of states and the challenges in understanding the vast degeneracy of quantum black hole states.  The Euclidean approach is extended to asymptotically AdS spacetime, where the Hawking-Page transition is discussed.

### Two Notions of Entropy

The author introduces von Neumann entropy, $S_N(\rho) = -\text{Tr}\,\rho\log\rho$, as a microscopic measure of entropy, contrasting it with thermodynamic entropy.  The relation $S_N \leq S_{th}$ is established, with equality for a maximally mixed state.  The author discusses the difficulty in distinguishing states of low von Neumann entropy from thermal equilibrium states, mentioning the Entanglement Thermalization Hypothesis.  The generalized entropy of a black hole is reinterpreted as a thermodynamic entropy, while von Neumann entropy is crucial for understanding ultraviolet divergences.  The author discusses the Page curve, which describes the entropy of an evaporating black hole, showing that the von Neumann entropy of the radiation and the black hole are approximately equal at all times, with a transition at the Page time.

### Black Holes and Von Neumann Entropy

The author discusses the interpretation of Bekenstein-Hawking entropy as entanglement entropy between regions inside and outside the horizon.  The generalized entropy is shown to be ultraviolet finite due to cancellations between divergences in the matter entropy and the renormalization of Newton's constant.  A sample computation of entanglement entropy in a two-dimensional CFT using the replica trick is presented, illustrating the ultraviolet divergence.  The author discusses the Bekenstein bound, showing how Casini's reinterpretation using relative entropy and Rindler wedges provides a rigorous version of the bound in quantum field theory.  The author analyzes R\'enyi entropies in the thermofield double state of a black hole, showing that they are well-defined due to cancellations between divergences in the matter entropy and the renormalization of Newton's constant.

### The Ryu-Takayanagi Formula

The Ryu-Takayanagi (RT) formula provides an area formula for von Neumann entropy in the context of AdS/CFT duality.  A heuristic motivation is presented using a pair of entangled black holes, arguing that the entanglement entropy is given by the area of an extremal surface homologous to the boundary region.  The original setting of the RT formula, involving a CFT on the conformal boundary of AdS spacetime, is discussed, with examples illustrating the formula's application to various regions and the existence of phase transitions.  A path integral argument for the RT formula is sketched, showing that the leading contribution to the entropy is given by the area of an extremal surface.  The author discusses quantum corrections to the RT formula, leading to the concept of quantum extremal surfaces and the interpretation of the entropy as the generalized entropy of the quantum extremal surface with minimum generalized entropy.  The implications of entanglement wedge reconstruction for the holographic map are discussed, highlighting the distributed and redundant encoding of bulk information in the boundary CFT.

### What is a White Hole?

The paper concludes with a discussion of white holes, the time-reversal of black holes.  Penrose diagrams are used to illustrate the causal structure of white hole formation.  The author argues that white hole formation represents a strong violation of the Second Law of Thermodynamics, requiring extreme fine-tuning of initial conditions.  The scrambling time of black holes is used to estimate the time scale over which a perturbation can significantly affect the white hole spacetime.  The author considers the quantum mechanical aspects of white holes, emphasizing the fine-tuning required for the incoming Hawking radiation.  The conclusion is that white hole spacetimes are highly unstable to perturbations acting sufficiently far in the past.

In [55]:
arxiv_id = extract_arxiv_id("https://arxiv.org/abs/2503.14446")
latex_source = fetch_arxiv_source(arxiv_id)
for key, value in latex_source.items():
    file_content = value

In [57]:
get_length = int(
    input("Type 0 for short summary, and 1 or 2 for long summary: "))

response = chains[get_length].invoke(
    {"Prompt": "Can you summarize the following? ", "Context": file_content})

# Access the text from the AIMessage object
summary = response.content if hasattr(
    response, 'content') else str(response)

display(Markdown(summary))

**1. Title & Authors:**

*Title:* Codimension One Foliations on Adjoint Varieties

*Author:* Crislaine Kuster


**2. Abstract:**

This paper classifies codimension-one foliations on adjoint varieties with mostly positive anti-canonical class.  For adjoint varieties with Picard number one, the foliations are shown to be induced by a pencil of hyperplane sections with respect to their minimal embedding.  Adjoint varieties with Picard number two exhibit a more complex structure, with multiple components of such foliations, each of which is described.  A novel concept, the degree of a foliation with respect to a family of rational curves, is introduced as a key tool for understanding these foliations and is of independent interest.


**3. Main Contributions:**

The paper's main contributions are:

* **Classification of codimension-one foliations:** The paper provides a classification of codimension-one foliations on adjoint varieties, distinguishing between those with Picard number one and two.  For Picard number one, it demonstrates that these foliations are always induced by pencils of hyperplane sections. For Picard number two, a more detailed description of the multiple components of such foliations is given.

* **Introduction of a new degree concept:** The paper introduces the concept of the degree of a codimension-one foliation with respect to a family of rational curves. This concept generalizes the classical definition of degree for foliations on projective spaces and provides a valuable tool for analyzing foliations on more general varieties.  The relationship between this degree and the normal sheaf of the foliation is established.

* **Analysis of foliations with degree zero:** The paper investigates codimension-one foliations with degree zero with respect to a minimal dominating family of rational curves.  It proves that such foliations are algebraically integrable on Fano varieties of Picard number one.  For adjoint varieties of Picard number one (excluding projective spaces), these foliations are shown to be pencils of hyperplane sections under the adjoint embedding.

* **Study of adjoint varieties with Picard number two:** The paper analyzes codimension-one foliations on adjoint varieties of Picard number two (hyperplane sections of  $\mathbb{P}^n \times \mathbb{P}^n$). It explores foliations with minimal degrees with respect to the two natural families of rational curves and shows that for $n \ge 3$, there are at least two irreducible components in the space of codimension-one foliations with degree zero with respect to both families. For $n=2$, a third component associated with a rigid foliation related to the affine Lie algebra is identified.


**4. Methodology:**

The paper employs a combination of techniques from algebraic geometry and the theory of holomorphic foliations.  Key methods include:

* **Harder-Narasimhan filtration:** This technique is used to analyze the stability of the tangent sheaf of the foliation.

* **Tangential foliation:** The tangential foliation is constructed to study the behavior of the foliation with respect to families of rational curves.  This involves analyzing the pullback of the tangent sheaf to rational curves and studying the resulting foliation on the space of morphisms from $\mathbb{P}^1$ to the variety.

* **Rational connected quotients:** The concept of rationally connected quotients is used to relate the behavior of the foliation to the geometry of the variety.

* **Transverse structures:** The paper analyzes transversely projective, transversely affine, and transversely Euclidean structures of foliations to understand their integrability properties.

* **Bott-Borel-Weil Theorem:** This theorem is applied to analyze homogeneous vector bundles on adjoint varieties.


**5. Key Equations & Mathematical Insights:**

The degree of a codimension-one foliation $\mathcal{F}$ with respect to a family $H$ of rational curves is defined as:

$\mathrm{deg}_{H}\, \mathcal{F} = N_\mathcal{F} \cdot C - 2$

where $N_\mathcal{F}$ is the normal sheaf of $\mathcal{F}$ and $C$ is a general curve in $H$.  This equation is crucial for relating the degree of the foliation to its normal sheaf.  The paper extensively uses this equation to analyze foliations with degree zero.


**6. Results & Discussion:**

The paper establishes several key results:  Theorem 1 shows that codimension-one foliations with degree zero with respect to a minimal dominating family of rational curves are algebraically integrable on Fano varieties of Picard number one.  Theorem 2 extends this result to adjoint varieties of Picard number one (excluding projective spaces), showing that these foliations are pencils of hyperplane sections.  The analysis of adjoint varieties with Picard number two reveals a more complex structure, with multiple components of foliations, depending on the degree with respect to the two families of lines.  The paper carefully distinguishes between algebraically integrable foliations and those defined by closed rational 1-forms.  The existence of multiple transverse structures is used to deduce properties of the foliations.


**7. Conclusion & Implications:**

The paper provides a significant contribution to the classification of codimension-one foliations on adjoint varieties.  The introduction of the degree concept with respect to families of rational curves offers a powerful tool for future research on foliations on more general varieties.  The results highlight the interplay between the geometry of the variety (Picard number, minimal families of rational curves) and the properties of the foliations (degree, integrability).  The detailed analysis of adjoint varieties with Picard number two suggests that the classification problem becomes significantly more intricate in higher Picard number settings.  The identification of rigid foliations associated with the affine Lie algebra points to the richness of the moduli space of foliations even in relatively simple cases.

In [68]:
arxiv_id = extract_arxiv_id("https://arxiv.org/abs/cond-mat/0506438")
latex_source = fetch_arxiv_source(arxiv_id)
for key, value in latex_source.items():
    file_content = value
    print(type(file_content))
    print(len(file_content))

<class 'str'>
367876


In [70]:
get_length = int(
    input("Type 0 for short summary, and 1 or 2 for long summary: "))

response = chains[get_length].invoke(
    {"Prompt": "Can you summarize the following? ", "Context": file_content})

# Access the text from the AIMessage object
summary = response.content if hasattr(
    response, 'content') else str(response)

display(Markdown(summary))

**1. Title & Authors:** Anyons in an exactly solved model and beyond by Alexei Kitaev

**2. Abstract:** This paper presents an exact solution to a spin-1/2 system on a honeycomb lattice with anisotropic nearest-neighbor interactions.  The model is mapped to free fermions in a static Z₂ gauge field, revealing a phase diagram with gapped and gapless phases. The gapped phase exhibits Abelian anyons, while the gapless phase, when subjected to a magnetic field, transitions to a phase with non-Abelian anyons whose braiding rules mirror those of Ising model conformal blocks.  The paper also explores a general theory of free fermions with gapped spectra, characterized by a spectral Chern number.

**3. Main Contributions:** The paper's primary contribution is the exact solution of a 2D quantum spin model exhibiting both Abelian and non-Abelian anyonic phases.  It establishes a connection between the spectral Chern number (ν) of the fermionic system and the anyonic properties of the excitations.  Specifically, it shows that Abelian anyons appear when ν is even, and non-Abelian anyons when ν is odd, with 16 distinct types of vortex-fermion statistics depending on ν mod 16.  The Chern number also governs edge thermal transport.  The paper further provides a mathematical framework for analyzing anyons, including an elementary theory of Chern numbers for quasidiagonal matrices.

**4. Methodology:** The paper employs a Jordan-Wigner transformation to map the spin model to a system of Majorana fermions.  The resulting Hamiltonian is a quadratic form in Majorana operators, allowing for an exact solution.  The analysis involves Fourier transforms to determine the fermionic spectrum and perturbation theory to study the effective Hamiltonian in different parameter regimes.  The Chern number is calculated using a quasidiagonal matrix formalism.

**5. Results & Conclusion:** The model exhibits three distinct phases: two gapped phases (A) with Abelian anyons and a gapless phase (B) that becomes gapped in the presence of a magnetic field, exhibiting non-Abelian anyons.  The anyonic properties are shown to depend on the spectral Chern number ν mod 16, while ν itself determines the edge thermal transport.  The paper provides a comprehensive algebraic description of the anyonic properties in all 16 cases, highlighting the robustness of these topological features.